In [ ]:
import torch
from pathlib import Path
import mlflow

In [ ]:
from src.training.load_config import load_config

In [ ]:
run_config = load_config()
output = Path(run_config['script_path']) / f"{run_config['run_path']}.pt"
experiment = run_config['mlflow_experiment']
mlflow_path = run_config['mlflow_path']
rggb = True
mlflow.set_tracking_uri(f"file://{mlflow_path}")
mlflow.set_experiment(experiment)

In [ ]:
output

In [ ]:

RUN_ID = "10df0d1d6eba4c6b887086d69ab390a7"  
ARTIFACT_PATH = run_config['run_path']

model_uri = f"runs:/{RUN_ID}/{ARTIFACT_PATH}"

try:
    model = mlflow.pytorch.load_model(model_uri)
    model.eval()
    model = model.to('cpu')
    print(f"Model successfully loaded from MLflow URI: {model_uri}")
    

except Exception as e:
    print(f"Error loading model from MLflow: {e}")

In [ ]:
run_config['crop_size']

In [ ]:
in_channels = run_config['model_params']['in_channels']
out_channels = run_config['model_params']['out_channels']
cond_input = run_config['model_params']['cond_input']
crop_size = run_config['crop_size']

if run_config['model_params']['rggb']:
    input = torch.rand(1, in_channels, crop_size//2, crop_size//2)
else:
    input = torch.rand(1, in_channels, crop_size, crop_size)

cond = torch.rand(1, cond_input)
residual = torch.rand(1, out_channels, crop_size, crop_size)
traced_script_module = torch.jit.trace(model, (input, cond, residual))
traced_script_module.save(output)